In [15]:
#!import config/Settings.cs 

// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 0.23.230906.2-preview"
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planning.Sequential;
using Microsoft.SemanticKernel.Planning.Stepwise;
using System.Text.Json;
using System.Collections.Generic;

//Create Kernel builder
var builder = new KernelBuilder();

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

IKernel kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 0.23.230906.2-preview

In [16]:
#!import skills/Bank/ContaCorrente.cs 
#!import skills/Bank/Boleto.cs 
#!import skills/Bank/BankSkills.cs 
#!import skills/Email/Emailskills.cs 

var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills"); 

In [17]:
var contaCorrentPrincipal = new ContaCorrente("123", "eu@contoso.com");
contaCorrentPrincipal.Depositar(1000);

Dictionary<string, ContaCorrente> contasCorrentes = new Dictionary<string, ContaCorrente>() {
    { "André", new ContaCorrente("456", "andre@contoso.com") },
    { "João", new ContaCorrente("789", "joao@contoso.com") }
};


Boleto[] boletos = { Boleto.Criar("1", 450), Boleto.Criar("2", 150) , Boleto.Criar("3", 1500) };

var bankSkills = kernel.ImportSkill(new BankSkills(contaCorrentPrincipal, boletos, contasCorrentes), "BankSkills");

kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "SemanticSkills");
var emailSkills = kernel.ImportSkill(new EmailSkills(), "EmailSkills");


var planner = new StepwisePlanner(kernel);

In [18]:
// Create a plan for the ask
var ask = "Pagar todos os boletos e tranferir 300 para o João, mandando um comprovante por e-mail. Não deixe o saldo negativo, e retorne o saldo final e os boletos que foram pagos. Priorize os de maior valor.";
var plan = planner.CreatePlan(ask);

//Console.WriteLine("Plan:");
//Console.WriteLine(plan.ToJson(true));
// Execute the plan
var result = await plan.InvokeAsync();



var steps = JsonSerializer.Deserialize<List<SystemStep>>(result.Variables["stepsTaken"]);
Console.WriteLine("Steps taken:");
foreach (SystemStep step in steps)
{
    Console.WriteLine("---------");
    Console.WriteLine("OriginalResponse : " + step.OriginalResponse );
    Console.WriteLine("Thought: " + step.Thought);
    Console.WriteLine("Action: " + step.Action);
    step.ActionVariables.ToList().ForEach(v => Console.WriteLine("ActionVariable: " + v.Key + " = " + v.Value));
    Console.WriteLine("Observation: " + step.Observation);
    Console.WriteLine("FinalAnswer: " + step.FinalAnswer);
}

Console.WriteLine("Resposta:");
Console.WriteLine(result.Result);


Boletos Pendentes: 
Numero:1 - Valor:450
Numero:2 - Valor:150
Numero:3 - Valor:1500

Saldo: 1000
Pagar Boleto 1 chamado
Saldo: 550
Pagar Boleto 2 chamado
Saldo: 400
ObterContaRelacionada João chamado
Saldo: 400
Transferir 300 para 789 chamado
Enviar e-mail para: joao@contoso.com
Texto: Assunto: Comprovante de Transferência Bancária

Olá,

Espero que este e-mail encontre você bem.

Estou escrevendo para confirmar que uma transferência bancária foi realizada com sucesso. Os detalhes da transação são os seguintes:

- Destinatário: João
- Valor da Transferência: 300

Por favor, considere este e-mail como um comprovante oficial da transação. Se você tiver alguma dúvida ou precisar de mais informações, não hesite em entrar em contato.

Agradeço a sua atenção e estou à disposição para quaisquer esclarecimentos.

Atenciosamente,

[Seu Nome]
[Seu contato]
Saldo: 100
Steps taken:
---------
OriginalResponse : [THOUGHT]
Para resolver essa tarefa, precisamos seguir os seguintes passos:

1. Obter os